In [ ]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
df = pd.read_csv("GrammarandProductReviews.csv", engine='python')

In [ ]:
df.head()

,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


In [ ]:
df.columns

Index(['id', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'ean', 'keys',
       'manufacturer', 'manufacturerNumber', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title',
       'reviews.userCity', 'reviews.userProvince', 'reviews.username', 'upc'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71044 entries, 0 to 71043
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    71044 non-null  object 
 1   brand                 71044 non-null  object 
 2   categories            71044 non-null  object 
 3   dateAdded             71044 non-null  object 
 4   dateUpdated           71044 non-null  object 
 5   ean                   39065 non-null  object 
 6   keys                  71044 non-null  object 
 7   manufacturer          70903 non-null  object 
 8   manufacturerNumber    70841 non-null  object 
 9   name                  71044 non-null  object 
 10  reviews.date          70977 non-null  object 
 11  reviews.dateAdded     71044 non-null  object 
 12  reviews.dateSeen      71044 non-null  object 
 13  reviews.didPurchase   32158 non-null  object 
 14  reviews.doRecommend   60429 non-null  object 
 15  reviews.id         

In [ ]:
#reviews.text	             Drop Rows	            Core feature for sentiment analysis; 36 missing values (71,044 to 71,008) is negligible.
#reviews.title	             Fill with "No Title"	Provides secondary context for sentiment without losing the row.
#reviews.rating	             None	                Dataset is already complete (71,044 non-null).
#brand / name / categories	 None	                These core identifiers are complete (71,044 non-null).
#reviews.doRecommend	     Impute with Mode	    Useful for sentiment validation; fill missing values with the most frequent choice (True/False).
#reviews.didPurchase	     Fill with "Unknown"	Over 50% missing; treating as a separate category is safer than dropping or guessing.
#reviews.username	         Fill with "Anonymous"	Required for the recommendation system to group users.
#ean / upc / keys	         Drop Columns	        High sparsity or redundant for the specific goal of sentiment and recommendation.
#userCity               	 Drop Columns	        Extremely high null counts (90%+); not statistically useful for this scope."""

In [ ]:
df = df.dropna(subset=['reviews.text'])

df['reviews.title'] = df['reviews.title'].fillna('No Title') # Fill missing titles with a placeholder

df['reviews.username'] = df['reviews.username'].fillna('Anonymous') # Fill missing usernames with 'Anonymous' for recommendation grouping

/tmp/ipython-input-2279899539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews.title'] = df['reviews.title'].fillna('No Title') # Fill missing titles with a placeholder
/tmp/ipython-input-2279899539.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews.username'] = df['reviews.username'].fillna('Anonymous') # Fill missing usernames with 'Anonymous' for recommendation grouping


In [ ]:
# Impute missing recommendations with the mode
if not df['reviews.doRecommend'].empty:
    mode_val = df['reviews.doRecommend'].mode()[0]
    df['reviews.doRecommend'] = df['reviews.doRecommend'].fillna(mode_val)

/tmp/ipython-input-831752365.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['reviews.doRecommend'] = df['reviews.doRecommend'].fillna(mode_val)
/tmp/ipython-input-831752365.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews.doRecommend'] = df['reviews.doRecommend'].fillna(mode_val)


In [ ]:
df['reviews.didPurchase'] = df['reviews.didPurchase'].fillna('Unknown') # Fill purchase status with 'Unknown' (too many nulls to drop or guess)

# Drop columns that are too sparse or irrelevant
cols_to_drop = ['reviews.userCity', 'reviews.userProvince', 'ean', 'reviews.userProvince']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

/tmp/ipython-input-1639459353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews.didPurchase'] = df['reviews.didPurchase'].fillna('Unknown') # Fill purchase status with 'Unknown' (too many nulls to drop or guess)


In [ ]:
#Text Preprocessing
#Merge Text Fields: Combine reviews.title and reviews.text into a single full_review column to capture all sentiment signals.

#Cleaning: Convert to lowercase, remove punctuation, special characters, and numbers using Regex.

#Tokenization & Stopwords: Tokenize strings and remove common English stopwords (e.g., "the", "is", "at") that don't carry emotional weight.

#Lemmatization: Reduce words to their root form (e.g., "running" to "run") to normalize the vocabulary.

In [ ]:
df['full_review'] = df['reviews.title'] + " " + df['reviews.text']

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower() # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text) # Remove special characters and numbers
    # Tokenize, remove stopwords, and lemmatize
    tokens = text.split()
    cleaned = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(cleaned)

df['cleaned_review'] = df['full_review'].apply(clean_text)

In [ ]:
#Feature Engineering & Transformation
#Sentiment Labeling: Create a target variable sentiment where ratings 4-5 are "Positive", 3 is "Neutral", and 1-2 are "Negative".

#Category Parsing: The categories column contains comma-separated strings; split these into a list or use one-hot encoding for the top N categories.

#Vectorization: Transform cleaned text into numerical format using TF-IDF (for traditional ML) or Token IDs (for Deep Learning/Transformers).

#Normalization: Scale numerical features like reviews.numHelpful using a MinMaxScaler if they are used as weights in the recommendation engine.

In [ ]:
# Create Sentiment Labels: 1 (Positive: 4-5), 0 (Neutral: 3), -1 (Negative: 1-2)
def label_sentiment(df):
    df['sentiment'] = df['reviews.rating'].apply(lambda x: 'Positive' if x >= 4 else ('Neutral' if x == 3 else 'Negative'))

label_sentiment(df)


In [ ]:
df.columns

Index(['id', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'keys',
       'manufacturer', 'manufacturerNumber', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title',
       'reviews.username', 'upc', 'full_review', 'cleaned_review',
       'sentiment'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71008 entries, 0 to 71043
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   71008 non-null  object 
 1   brand                71008 non-null  object 
 2   categories           71008 non-null  object 
 3   dateAdded            71008 non-null  object 
 4   dateUpdated          71008 non-null  object 
 5   keys                 71008 non-null  object 
 6   manufacturer         70867 non-null  object 
 7   manufacturerNumber   70806 non-null  object 
 8   name                 71008 non-null  object 
 9   reviews.date         70941 non-null  object 
 10  reviews.dateAdded    71008 non-null  object 
 11  reviews.dateSeen     71008 non-null  object 
 12  reviews.didPurchase  71008 non-null  object 
 13  reviews.doRecommend  71008 non-null  bool   
 14  reviews.id           32156 non-null  float64
 15  reviews.numHelpful   32506 non-null  floa

In [ ]:
df[['cleaned_review', 'sentiment', 'reviews.id', 'reviews.numHelpful']].head()

,cleaned_review,sentiment,reviews.id,reviews.numHelpful
0,awesome love album good hip hop side current p...,Positive,NaN,0.0
1,good good flavor review collected part promotion,Positive,100209113.0,NaN
2,good good flavor,Positive,100209113.0,NaN
3,disappointed read review looking buying one co...,Negative,113026909.0,NaN
4,irritation husband bought gel u gel caused irr...,Negative,171267657.0,NaN


In [ ]:
df['reviews.numHelpful'].nunique()

72

In [ ]:
df['reviews.id'].nunique()

31372

In [ ]:
print(df['reviews.username'].nunique())
print('-'*20)
print(df['reviews.username'].value_counts())
print('-'*20)
print(df['reviews.username'].isna().sum())


58051
--------------------
reviews.username
An anonymous customer    421
Anonymous                215
Mike                     105
Chris                     88
ByAmazon Customer         58
                        ... 
ByP                        1
ByAuctionmail20022000      1
ByABenn                    1
ByPatricia Wilson          1
ByAmanda K                 1
Name: count, Length: 58051, dtype: int64
--------------------
0


In [ ]:
df.drop(columns=['full_review'], inplace=True)


In [ ]:
df.columns

Index(['id', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'keys',
       'manufacturer', 'manufacturerNumber', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title',
       'reviews.username', 'upc', 'cleaned_review', 'sentiment'],
      dtype='object')

In [ ]:
#Sentiment Analysis Model
#Approach: Use a Transformer-based model (like BERT or RoBERTa) or a fast approach like VADER/TextBlob for initial labeling.

#Refinement: Fine-tune a classifier to predict the sentiment score. This will be used later to "weight" the recommendations.

#Brand & Category Affinity Mapping
#Aggregation: Group data by brand and categories.

#Metric: Calculate a "Brand Health Score" combining average rating and average sentiment polarity.

#Output: Insights on which brands are trending upward vs. downward.

#Sentiment-Aware Recommender System
#Collaborative Filtering: Use reviews.username and id (product ID) to create a user-item matrix.

#The "Sentiment Filter": This is the core requirement. If your algorithm suggests "Product A," but "Product A" has a recent spike in negative sentiment, the system should penalize its rank and suggest "Product B" instead.

In [ ]:
# Create Baseline Model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

X = df['cleaned_review']
y = df['sentiment']

# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Vectorization (TF-IDF) Converting text into numerical vectors
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Model Training Logistic Regression is a strong, fast baseline for text classification
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Evaluation
y_pred = model.predict(X_test_tfidf)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.9133924799324039
              precision    recall  f1-score   support

    Negative       0.83      0.65      0.73      1107
     Neutral       0.56      0.21      0.30       873
    Positive       0.93      0.99      0.96     12222

    accuracy                           0.91     14202
   macro avg       0.77      0.62      0.66     14202
weighted avg       0.90      0.91      0.90     14202



In [ ]:
# Drop columns not useful for sentiment analysis
data = df.drop(columns=['reviews.title', 'reviews.text', # Redundant information, now in cleaned_review
                        'dateAdded', # reviews.date is more accurate for analyzing consumer trends
                        'dateUpdated', # Not relevant for sentiment or recommendation
                        'keys', # Redundant unique identifier
                        'upc', # Product code not needed for text analysis
                        'manufacturerNumber', # Redundant
                        'reviews.sourceURLs']) # Irrelevant for tet analysis

data.columns

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.username', 'cleaned_review', 'sentiment'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71008 entries, 0 to 71043
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   71008 non-null  object 
 1   brand                71008 non-null  object 
 2   categories           71008 non-null  object 
 3   manufacturer         70867 non-null  object 
 4   name                 71008 non-null  object 
 5   reviews.date         70941 non-null  object 
 6   reviews.dateAdded    71008 non-null  object 
 7   reviews.dateSeen     71008 non-null  object 
 8   reviews.didPurchase  71008 non-null  object 
 9   reviews.doRecommend  71008 non-null  bool   
 10  reviews.id           32156 non-null  float64
 11  reviews.numHelpful   32506 non-null  float64
 12  reviews.rating       71008 non-null  int64  
 13  reviews.username     71008 non-null  object 
 14  cleaned_review       71008 non-null  object 
 15  sentiment            71008 non-null  obje

In [ ]:
data.head()

,id,brand,categories,manufacturer,name,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.username,cleaned_review,sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,2018-02-04T07:28:12Z,"2018-01-15T04:45:00.000Z,2018-01-18T14:30:00.0...",Unknown,True,NaN,0.0,5,Joshua,awesome love album good hip hop side current p...,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,2017-09-23T02:53:06Z,2017-09-16T09:45:00.000Z,True,True,100209113.0,NaN,5,Dorothy W,good good flavor review collected part promotion,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,2017-09-06T04:49:31Z,2017-08-23T10:37:00.000Z,True,True,100209113.0,NaN,5,Dorothy W,good good flavor,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,2017-09-11T17:13:57Z,"2017-09-04T12:18:00.000Z,2017-08-09T02:12:00.000Z",False,False,113026909.0,NaN,1,Rebecca,disappointed read review looking buying one co...,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,2017-09-11T17:13:57Z,"2017-09-04T12:18:00.000Z,2017-08-09T02:12:00.000Z",False,False,171267657.0,NaN,1,Walker557,irritation husband bought gel u gel caused irr...,Negative


In [ ]:
# IOB Tagging

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification

# Define Label Map
label_list = ["O", "B-BRAND", "I-BRAND", "B-CAT", "I-CAT"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

def generate_iob_tags(row):
    tokens = str(row['cleaned_review']).split()
    brand_tokens = str(row['brand']).lower().split()
    cat_tokens = str(row['categories']).lower().replace(',', ' ').split()

    tags = ["O"] * len(tokens)
    # Simple matching logic for B- and I- tags
    for i in range(len(tokens)):
        # Brand Match
        if tokens[i:i+len(brand_tokens)] == brand_tokens:
            tags[i] = "B-BRAND"
            for j in range(1, len(brand_tokens)):
                tags[i+j] = "I-BRAND"
        # Category Match
        elif tokens[i] in cat_tokens:
            tags[i] = "B-CAT"
    return tags

data['iob_tags'] = df.apply(generate_iob_tags, axis=1)

In [ ]:
data.columns

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.username', 'cleaned_review', 'sentiment',
       'iob_tags'],
      dtype='object')

In [ ]:
data['iob_tags'].head()

,iob_tags
0,"[O, O, O, O, O, O, O, O, B-CAT, O, O, O, O, O,..."
1,"[O, O, O, O, O, O, O]"
2,"[O, O, O]"
3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [ ]:
# We must align our IOB tags so only the first sub-token gets the tag, and others get a ignore index
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class NERDataset(Dataset):
    def __init__(self, df, tokenizer, label2id, max_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = row['cleaned_review'].split()
        word_labels = row['iob_tags']

        encoding = self.tokenizer(text, is_split_into_words=True,
                                  truncation=True, padding='max_length',
                                  max_length=self.max_len)

        labels = []
        word_ids = encoding.word_ids()
        for word_idx in word_ids:
            if word_idx is None:
                labels.append(-100) # Ignore special tokens
            else:
                # Assign tag to the first sub-token, -100 to others
                labels.append(self.label2id[word_labels[word_idx]])

        return {
            'input_ids': torch.tensor(encoding['input_ids']),
            'attention_mask': torch.tensor(encoding['attention_mask']),
            'labels': torch.tensor(labels)
        }

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from torch.optim import AdamW
# Initialize Dataset and DataLoader
dataset = NERDataset(data, tokenizer, label2id)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Load Pre-trained BERT
model = BertForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(5):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item()}")

# Save Model
model.save_pretrained("./bert_ner_model")
tokenizer.save_pretrained("./bert_ner_model")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 Loss: 0.02034280262887478
Epoch 2 Loss: 0.08969680964946747
Epoch 3 Loss: 0.004815481137484312
Epoch 4 Loss: 0.02758599817752838
Epoch 5 Loss: 0.0003210654540453106


('./bert_ner_model/tokenizer_config.json',
 './bert_ner_model/special_tokens_map.json',
 './bert_ner_model/vocab.txt',
 './bert_ner_model/added_tokens.json',
 './bert_ner_model/tokenizer.json')

In [ ]:
def extract_entities(text, model, tokenizer):
    # Prepare input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the highest probability label for each token
    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Filter out special tokens ([CLS], [SEP], [PAD])
    extracted_entities = []
    for token, pred in zip(tokens, predictions[0]):
        label = id2label[pred.item()]
        if label != "O":
            extracted_entities.append((token, label))

    return extracted_entities

# Example Usage:
new_review = "I think Lundberg rice cakes are much better than generic brands."
entities = extract_entities(new_review, model, tokenizer)
print(entities)


[('lund', 'B-BRAND'), ('##berg', 'B-BRAND'), ('rice', 'B-CAT'), ('[SEP]', 'B-CAT')]


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification

# Load model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForTokenClassification.from_pretrained("./bert_ner_model").to(device)
tokenizer = BertTokenizerFast.from_pretrained("./bert_ner_model")

def bulk_extract_entities(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    entities = []
    for token, pred in zip(tokens, predictions[0]):
        label = id2label[pred.item()]
        if label != "O" and not token.startswith("##"):
            entities.append(token)
    return list(set(entities)) # Return unique mentions

# Apply to entire dataframe
data['extracted_brands'] = data['cleaned_review'].apply(bulk_extract_entities)
data.to_csv('inference_extraction.csv', index=False)

In [ ]:
data.columns

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.username', 'cleaned_review', 'sentiment',
       'iob_tags', 'extracted_brands'],
      dtype='object')

In [ ]:
data.head()

,id,brand,categories,manufacturer,name,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.username,cleaned_review,sentiment,iob_tags,extracted_brands
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,2018-02-04T07:28:12Z,"2018-01-15T04:45:00.000Z,2018-01-18T14:30:00.0...",Unknown,True,NaN,0.0,5,Joshua,awesome love album good hip hop side current p...,Positive,"[O, O, O, O, O, O, O, O, B-CAT, O, O, O, O, O,...","[pop, [SEP]]"
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,2017-09-23T02:53:06Z,2017-09-16T09:45:00.000Z,True,True,100209113.0,NaN,5,Dorothy W,good good flavor review collected part promotion,Positive,"[O, O, O, O, O, O, O]",[[SEP]]
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,2017-09-06T04:49:31Z,2017-08-23T10:37:00.000Z,True,True,100209113.0,NaN,5,Dorothy W,good good flavor,Positive,"[O, O, O]",[[SEP]]
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,2017-09-11T17:13:57Z,"2017-09-04T12:18:00.000Z,2017-08-09T02:12:00.000Z",False,False,113026909.0,NaN,1,Rebecca,disappointed read review looking buying one co...,Negative,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[SEP], personal]"
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,2017-09-11T17:13:57Z,"2017-09-04T12:18:00.000Z,2017-08-09T02:12:00.000Z",False,False,171267657.0,NaN,1,Walker557,irritation husband bought gel u gel caused irr...,Negative,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",[[SEP]]


In [ ]:
"""from transformers import BertForSequenceClassification, BertTokenizerFast
import torch.nn.functional as F

# Load a Sentiment model (this one outputs 5 labels corresponding to 1-5 stars)
sent_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sent_tokenizer = BertTokenizerFast.from_pretrained(sent_model_name)
sent_model = BertForSequenceClassification.from_pretrained(sent_model_name).to(device)

def get_sentiment_weights(df, model, tokenizer, device):
    model.eval()
    sentiment_weights = []

    for text in tqdm(df['cleaned_review'], desc="Calculating Sentiment Weights"):
        # Ensure text is a string
        inputs = tokenizer(str(text), return_tensors="pt", truncation=True,
                           padding=True, max_length=128).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            # Sequence Classification logits are (batch_size, num_labels)
            # For 5-class sentiment, probs will be a vector of 5 probabilities
            probs = F.softmax(outputs.logits, dim=1)

            # 2. CALCULATION LOGIC:
            # If the model has 5 classes (1-5 stars), we calculate a weighted average
            # [1*P(1) + 2*P(2) + 3*P(3) + 4*P(4) + 5*P(5)] / 5 to get a 0.0 - 1.0 weight
            weights = torch.tensor([0.2, 0.4, 0.6, 0.8, 1.0]).to(device)
            final_weight = torch.sum(probs[0] * weights).item()

            sentiment_weights.append(final_weight)

    df['sentiment_weight'] = sentiment_weights
    return df"""

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification
from tqdm import tqdm
import torch.nn.functional as F

In [ ]:
class SentimentReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len=128):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

#Initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "nlptown/bert-base-multilingual-uncased-sentiment" #pre-trained BERT model specifically fine-tuned for sentiment analysis
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
def run_batch_sentiment(df, model, tokenizer, batch_size=32):
    model.eval()
    dataset = SentimentReviewDataset(df['cleaned_review'].values, tokenizer)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=2)

    weights = []
    # Weighted average scale for 1-5 star model: [1*, 2*, 3*, 4*, 5*]
    scale = torch.tensor([0.2, 0.4, 0.6, 0.8, 1.0]).to(device)

    with torch.no_grad():
        for batch in tqdm(loader, desc="Processing Batches"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1) # Shape: [batch_size, 5]

            # weighted average for each review in batch
            batch_weights = torch.matmul(probs, scale)
            weights.extend(batch_weights.cpu().tolist())

    df['sentiment_weight'] = weights
    return df

data = run_batch_sentiment(data, model, tokenizer, batch_size=64)

Processing Batches: 100%|██████████| 1110/1110 [07:30<00:00,  2.46it/s]


In [ ]:
data.to_csv('sentiment_score.csv', index=False)

In [ ]:
data['sentiment_weight'].head(10)

,sentiment_weight
0,0.879193
1,0.808363
2,0.797752
3,0.303378
4,0.315339
5,0.247649
6,0.489007
7,0.555260
8,0.302616
9,0.231114


In [ ]:
data.columns

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.username', 'cleaned_review', 'sentiment',
       'iob_tags', 'extracted_brands', 'sentiment_weight'],
      dtype='object')

To calculate the final Brand Health Score, we will use a weighted composite formula that combines the unstructured signal from BERT with the structured signals provided in the dataset.

The Brand Health Score Formula

$$Brand\ Health\ Score = \left( \overline{S}_{weight} \times 0.5 \right) + \left( \frac{\overline{R}}{5} \times 0.3 \right) + \left( \overline{Rec} \times 0.2 \right)$$

Where:

$\overline{S}_{weight}$ : The average BERT Sentiment Weight for the brand (0 to 1).

$\overline{R}$ : The average Numerical Rating for the brand (scale 1-5).

$\overline{Rec}$ : The average Recommendation Rate (percentage of "True" in reviews.doRecommend).

This formula is designed to be "sentiment-aware" rather than just relying on simple averages:

Nuance vs. Noise: Star ratings are often extreme (either 1 or 5). By giving 50% weight to BERT Sentiment, we prioritize the actual emotional nuance found in the text, which is more reliable than a simple click.

Normalization: By dividing the rating by 5 and using the probability of recommendation (0 to 1), all three signals are placed on the same scale, ensuring no single metric accidentally overpowers the others.

Consumer Intent: The reviews.doRecommend column acts as a final "loyalty" signal. A brand might have good sentiment, but if users aren't recommending it, the brand health is technically lower.

Bias Correction: If a user gives 5 stars but writes a negative review, the BERT weight will pull the score down, preventing "false positives" in your recommendation engine.

In [ ]:
#Step 1: It finds the average BERT Sentiment Weight for that brand.

#Step 2: It finds the average Numerical Rating (out of 5) for that brand.

#Step 3: It finds the percentage of reviews where the user said "Yes, I recommend this".

#Step 4: It applies the weighted formula to combine these into one final Health Score.

In [ ]:
"""# Clean the 'reviews.doRecommend' column
# We convert it to 1s and 0s so we can calculate a mean (percentage)
data['reviews.doRecommend'] = data['reviews.doRecommend'].astype(float)

# Group by Brand and aggregate the three signals
brand_health_df = data.groupby('brand').agg({
    'sentiment_weight': 'mean',
    'reviews.rating': 'mean',
    'reviews.doRecommend': 'mean'
}).reset_index()

# Calculate the Final Weighted Brand Health Score (scaled 0-100)
brand_health_df['health_score'] = (
    (brand_health_df['sentiment_weight'] * 0.5) +
    ((brand_health_df['reviews.rating'] / 5) * 0.3) +
    (brand_health_df['reviews.doRecommend'] * 0.2)
) * 100

# Display the results
print(brand_health_df.sort_values(by='health_score', ascending=False).head(10))"""

In [ ]:
def calculate_brand_health(df):
    """
    Aggregates review data to calculate a weighted Brand Health Score.

    Formula: (Sentiment * 0.5) + (Rating/5 * 0.3) + (Recommendation Rate * 0.2)
    """
    # We convert it to 1s and 0s so we can calculate a mean (percentage)
    temp_df = df.copy()
    temp_df['reviews.doRecommend'] = pd.to_numeric(temp_df['reviews.doRecommend'], errors='coerce').fillna(0).astype(float)

    # Aggregate data by brand
    # We take the mean of our three key signals
    brand_stats = temp_df.groupby('brand').agg({
        'sentiment_weight': 'mean',
        'reviews.rating': 'mean',
        'reviews.doRecommend': 'mean'
    }).reset_index()

    # Calculate the Weighted Health Score (Scale 0-100)
    # We normalize ratings by dividing by 5 to keep all inputs between 0 and 1
    brand_stats['health_score'] = (
        (brand_stats['sentiment_weight'] * 0.5) +
        ((brand_stats['reviews.rating'] / 5) * 0.3) +
        (brand_stats['reviews.doRecommend'] * 0.2)
    ) * 100

    # Round for clean presentation and sort by health
    brand_stats['health_score'] = brand_stats['health_score'].round(2)
    return brand_stats.sort_values(by='health_score', ascending=False)

In [ ]:
brand_health_lookup = calculate_brand_health(data)

In [ ]:
brand_health_lookup.to_csv('brand_health_lookup.csv', index=False)

In [ ]:
brand_health_lookup

,brand,sentiment_weight,reviews.rating,reviews.doRecommend,health_score
297,Season Seafood,0.975468,5.0,1.0,98.77
154,JNH Lifestyles,0.974536,5.0,1.0,98.73
107,Fangio Lighting,0.973263,5.0,1.0,98.66
156,Jake And The Neverland Pirates,0.970264,5.0,1.0,98.51
179,La Tortilla Factory,0.970062,5.0,1.0,98.50
...,...,...,...,...,...
116,Gone Fishing,0.239665,1.0,1.0,37.98
136,Horizon,0.228351,1.0,1.0,37.42
358,Udi's,0.213134,1.0,1.0,36.66
234,Nordicware,0.423815,1.0,0.0,27.19


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def predict_brand_trends(df, time_range='M'):
    """
    Predicts brand health trends using modular time ranges and robust date parsing.
    """
    # We use format='ISO8601' to handle "2016-09-13T00:00:00Z"
    # utc=True handles the 'Z' (Zulu/UTC) time offset
    # df['reviews.date'] = pd.to_datetime(df['reviews.date'], format='ISO8601', utc=True)
    df['reviews.date'] = pd.to_datetime(df['reviews.date'], format='mixed', errors='coerce', utc=True)

    # DROPPING: Remove rows where the date was actually text or null
    df = df.dropna(subset=['reviews.date'])

    # Create Time Buckets
    df['time_bucket'] = df['reviews.date'].dt.to_period(time_range)

    # et history by applying calculate_brand_health function
    periodic_history = df.groupby('time_bucket').apply(calculate_brand_health).reset_index()

    trend_results = []

    # Calculate the regression slope for each brand
    for brand in periodic_history['brand'].unique():
        brand_data = periodic_history[periodic_history['brand'] == brand].sort_values('time_bucket')

        if len(brand_data) >= 2:
            X = np.array(range(len(brand_data))).reshape(-1, 1)
            y = brand_data['health_score'].values

            model = LinearRegression().fit(X, y)
            slope = model.coef_[0]

            # Trend Classification
            if slope > 0.5:
                status = "Upward"
            elif slope < -0.5:
                status = "Downward"
            else:
                status = "Stable"

            trend_results.append({
                'brand': brand,
                'slope': round(slope, 4),
                'trend_status': status,
                'recent_health': y[-1]
            })

    return pd.DataFrame(trend_results)


In [ ]:
brand_trends_df = predict_brand_trends(data, time_range='M')

/tmp/ipython-input-1188058370.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['time_bucket'] = df['reviews.date'].dt.to_period(time_range)
/tmp/ipython-input-1188058370.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_bucket'] = df['reviews.date'].dt.to_period(time_range)
/tmp/ipython-input-1188058370.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  periodic_history = df.groupby('time_

In [ ]:
brand_trends_df.to_csv('brand_trend_monthly.csv', index=False)

In [ ]:
brand_trends_df

,brand,slope,trend_status,recent_health
0,Almay,1.0697,Upward,85.25
1,Lundberg,-0.0272,Stable,54.32
2,Target.com Use Only,0.0608,Stable,88.56
3,Warner Bros.,0.0147,Stable,96.83
4,CRITERION COLLECTION INC,-0.0051,Stable,97.27
...,...,...,...,...
310,Paramount Pictures,-0.5750,Downward,89.48
311,Maple Grove Farms of Vermont,-1.5800,Downward,88.68
312,Ben & Jerry's,3.0643,Upward,98.96
313,Haagen-Dazs,21.3200,Upward,90.82


**The Data Ingestion Plan**

We will use three distinct data sources created throughout this project:

User-Item Interaction Data: From your main data DataFrame, you will use reviews.username (User), id (Product), and reviews.rating (Interaction Signal).

Brand Health Scores: The brand_health_lookup DataFrame, which provides a static snapshot of a brand's overall quality (0-100).

Brand Trends: The brand_trends_df DataFrame, which provides the dynamic "momentum" (Upward/Downward) and the slope of that change.

**The Recommendation Engine Architecture**

The engine will follow a Two-Stage Re-ranking Architecture:Stage

1: Candidate Generation (Collaborative Filtering)Method: Matrix Factorization (using the SVD algorithm from the Surprise library) or a simple User-Item similarity matrix.

Goal: Predict how much a user will like a product based on how other similar users rated it.

Output: A list of "Raw Recommendations" with predicted ratings (e.g., User A $\rightarrow$ Product X: 4.2 stars).

Stage 2: Sentiment-Aware Re-ranking (The Hybrid Layer)This is where your BERT and Regression work come into play. You will apply a Weighting Formula to adjust the raw scores:$$Hybrid\ Score = (CF\ Score \times 0.6) + (Health\ Score\ Factor \times 0.3) + (Trend\ Boost \times 0.1)$$Health Score Factor: If a product belongs to a brand with a 95/100 health score, its rank is boosted.

Trend Boost: If the trend_status is "Upward," the product gets a bonus. If "Downward," it is penalized.

In [ ]:
"""!pip install scikit-surprise
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_factory import get_dataset # If using standard datasets

# Prepare data for the Surprise library
# We need User, Product ID, and Rating
reader = Reader(rating_scale=(1, 5))
cf_data = Dataset.load_from_df(data[['reviews.username', 'id', 'reviews.rating']], reader)
trainset = cf_data.build_full_trainset()

# Train the SVD Model
svd_model = SVD()
svd_model.fit(trainset)"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2555928 sha256=5929ecfea1156d3f9455d93305d75bb46d32e7dbb269da4d1ba492a2d8c89405
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [ ]:
!pip install "numpy<2" --force-reinstall
import os
os.kill(os.getpid(), 9)

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.0
    Uninstalling numpy-2.4.0:
      Successfully uninstalled numpy-2.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.

In [ ]:
data = pd.read_csv('sentiment_score.csv')

In [ ]:
import numpy as np # Must be imported before surprise in many environments
import pandas as pd
from surprise import SVD, Dataset, Reader

clean_cf_data = data.dropna(subset=['reviews.username', 'id', 'reviews.rating']).copy()
reader = Reader(rating_scale=(1, 5))
cf_data = Dataset.load_from_df(clean_cf_data[['reviews.username', 'id', 'reviews.rating']], reader)
trainset = cf_data.build_full_trainset()

svd_model = SVD(n_factors=50, random_state=42)

svd_model.fit(trainset)

In [ ]:
brand_health_lookup = pd.read_csv('brand_health_lookup.csv')
brand_trends_df = pd.read_csv('brand_trend_monthly.csv')

In [ ]:
def get_personalized_hybrid_recommendations(user_id, df, cf_model, brand_health, brand_trends, top_n=10):
    """
    Hybrid Recommender with 80% weight on Personal Preference (CF).
    """
    all_prods = df[['id', 'name', 'brand']].drop_duplicates()

    # Check if user is known to the model
    is_known = user_id in trainset._raw_user_names

    if not is_known:
        # For new users, we must use Global Health/Trend logic
        global_rank = brand_health.merge(brand_trends, on='brand')
        global_rank = global_rank.sort_values(by=['health_score', 'slope'], ascending=False)
        best_brands = global_rank['brand'].head(top_n).tolist()
        return all_prods[all_prods['brand'].isin(best_brands)].head(top_n)

    recommendations = []

    for _, row in all_prods.iterrows():
        p_id = row['id']
        b_name = row['brand']

        # A. Get CF Prediction (0-5 scale)
        cf_score = cf_model.predict(user_id, p_id).est

        # B. Get Brand Health (0-100 scale)
        h_info = brand_health.loc[brand_health['brand'] == b_name, 'health_score'].values
        final_health = h_info[0] if len(h_info) > 0 else 50

        # C. Get Trend Slope
        t_info = brand_trends[brand_trends['brand'] == b_name]
        slope = t_info['slope'].values[0] if not t_info.empty else 0

        # D. NEW WEIGHTED FORMULA
        # 80% Personal Preference | 15% Brand Health | 5% Trend Momentum
        final_score = (cf_score / 5 * 0.8) + (final_health / 100 * 0.15) + (slope * 0.05)

        recommendations.append({
            'product_name': row['name'],
            'brand': b_name,
            'hybrid_score': round(final_score * 100, 2),
            'trend': t_info['trend_status'].values[0] if not t_info.empty else "Stable"
        })

    return pd.DataFrame(recommendations).sort_values(by='hybrid_score', ascending=False).head(top_n)

In [ ]:
results = get_hybrid_recommendations(test_user, data, svd_model, brand_health_lookup, brand_trends_df)
print(results)

                                          product_name             brand  \
346  John Frieda Brilliant Brunette Colour Protecti...       John Frieda   
305         Swann Door Alarm - White (swads-Alarmk-Gl)             Swann   
326               Haagen-Dazs Gelato, Sea Salt Caramel       Haagen-Dazs   
487  Arched Petrizzi Decorative Wall Mirror Silver ...         Uttermost   
18   Craft Punch Giga Scallop Circle 45 24687534 To...          No Brand   
253  Ceiling Fan With Light White 14.2 X 29.9 X 9.2...        Hunter Fan   
111              Charcoal Stone Toothbrush Holder Gray          No Brand   
58              Wagan Smartac 80watt Inverter With Usb             Wagan   
306                  Ust 30-Day Duro Lantern, Titanium               Ust   
516  Ge Led 60watt Outdoor Post Light Bulb - Soft W...  General Electric   

     hybrid_score   trend  
346        4.8763  Upward  
305        3.2025  Upward  
326        2.8918  Upward  
487        2.6374  Upward  
18         2.4728  Upwa

- Collaborative Filtering Model is identifying multi-category interests for Dorothy W. Even with diverse items, the system maintains high quality

- It has successfully filtered out brands with "Stable" or "Downward" trends, ensuring Dorothy W is only suggested products from brands gaining consumer favor.

- Products like "Craft Punch Giga Scallop..." associated with "No Brand" still appear with an Upward trend.This suggests that the BERT Sentiment Task detected positive sentiment in the reviews for these specific items. Even without a major brand name, the engine identifies them as high-quality based on the raw sentiment_weight generated from the reviews.





In [ ]:
def get_and_visualize_user_recommendations(data, cf_model, brand_health, brand_trends):
    """
    Asks for a username, generates hybrid recommendations, and visualizes the results.
    """
    target_user = input("Please enter the username for recommendations: ")

    results_df = get_hybrid_recommendations(target_user, data, cf_model, brand_health, brand_trends)

    if results_df.empty:
        print(f"No recommendations found for user: {target_user}")
        return

    viz_df = results_df.sort_values(by='hybrid_score', ascending=True)

    plt.figure(figsize=(12, 8))

    colors = ['#27ae60' if t == 'Upward' else '#95a5a6' for t in viz_df['trend']]

    bars = plt.barh(viz_df['product_name'], viz_df['hybrid_score'], color=colors)

    plt.xlabel('Hybrid Match Score (Scale 0-5)', fontsize=12, fontweight='bold')
    plt.ylabel('Recommended Products', fontsize=12, fontweight='bold')
    plt.title(f'Sentiment-Aware Recommendations for: {target_user}', fontsize=15, pad=20)
    plt.grid(axis='x', linestyle='--', alpha=0.6)

    for bar in bars:
        width = bar.get_width()
        plt.text(width + 0.05, bar.get_y() + bar.get_height()/2,
                 f'{width:.2f}', va='center', fontsize=10, fontweight='bold')

    plt.tight_layout()
    plt.show()